In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/basededados/vendas_modificado5.csv


In [2]:
df = pd.read_csv ('/kaggle/input/basededados/vendas_modificado5.csv')

In [3]:
#instalando as bibliotecas necessarias

!pip install mlxtend

from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import os

INFO: pip is looking at multiple versions of mlxtend to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement scikit-learn>=1.3.1 (from mlxtend) (from versions: none)
ERROR: No matching distribution found for scikit-learn>=1.3.1


In [4]:
#criando a "cesta" para comecar a analise, onde é id_da_compra x produto

cesta = df.groupby(['id_da_compra', 'produto'])['quantidade'].sum().unstack().fillna(0)

cesta_binaria = cesta.applymap(lambda x: 1 if x > 0 else 0)

/tmp/ipykernel_31/3138919918.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cesta_binaria = cesta.applymap(lambda x: 1 if x > 0 else 0)


In [5]:
#aplicando apriori para gerar o itemsets frequentes
itemsets = apriori(cesta_binaria, min_support=0.01, use_colnames=True)

/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [8]:
#agora sim, podemos gerar as regras com uma confianca minima determinada
#onde o antecendets ( produto comprado ), consequents (produdo que costuma vir acompanhado), 
#suport ( frequência dessa combinação), confidence (probabilidade), lift (forca da associação) 
regras = association_rules(itemsets, metric="confidence", min_threshold=0.3)
regras[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values(by='lift', ascending=False).head(10)

,antecedents,consequents,support,confidence,lift
155,(Carvão),"(Cerveja, Refrigerante)",0.014550,0.612202,9.464444
157,"(Suco De Laranja, Carvão)",(Cerveja),0.012784,0.997399,8.957185
328,"(Açúcar, Arroz)","(Feijão, Molho De Tomate)",0.010784,0.471574,8.953794
154,"(Carvão, Refrigerante)",(Cerveja),0.014550,0.995439,8.939582
15,(Carvão),(Cerveja),0.023634,0.994390,8.930160
158,(Carvão),"(Cerveja, Suco De Laranja)",0.012784,0.537868,8.724399
335,"(Açúcar, Molho De Tomate)","(Feijão, Café)",0.016184,0.321736,8.097254
333,"(Feijão, Café)","(Açúcar, Molho De Tomate)",0.016184,0.407299,8.097254
327,"(Feijão, Arroz)","(Açúcar, Molho De Tomate)",0.010784,0.397176,7.896015
336,"(Café, Molho De Tomate)","(Feijão, Açúcar)",0.016184,0.389334,6.822335


In [10]:
#quero ver se o carvao teve uma boa compra no ano em que ocorreu mais compras
#porem para isso preciso descubrir qual o ano que mais teve vendas


df['data'] = pd.to_datetime(df['data'])


df['ano'] = df['data'].dt.year


vendas_por_ano = df.groupby('ano')['quantidade'].sum().sort_values(ascending=False)

print(vendas_por_ano)

ano
2018    577401
2020    536542
2021    524734
2022    503419
2019    481348
Name: quantidade, dtype: int64


In [11]:
#vamos ver os produtos que mais venderam no ano que teve maior vendas
df['data'] = pd.to_datetime(df['data'])
df_2018 = df[df['data'].dt.year == 2018]
ranking_2018 = df_2018.groupby('produto')['quantidade'].sum().sort_values(ascending=False)
print(ranking_2018.head(10))

produto
Pasta De Dente      47118
Sabonete            45108
Queijo Mussarela    44711
Manteiga            42389
Desinfetante        33372
Papel Toalha        32450
Cerveja             26647
Molho De Tomate     21125
Condicionador       20025
Vinho               17316
Name: quantidade, dtype: int64


Nota se que o carvao nao esta no top 10 produtos mais vendidos no ano em que ocorreu mais vendas, porem o seu consequents está, que é a cerveja, logo deixar os dois próximos pode ser uma boa estratégia para alavancar a venda de ambos.

In [12]:
#agora vamos ver se no geral a cerveja ainda se mantem no top 10

ranking_produtos = df.groupby('produto')['quantidade'].sum().sort_values(ascending=False)
ranking_df = ranking_produtos.reset_index().rename(columns={'quantidade': 'total_vendido'})
ranking_df['rank'] = ranking_df['total_vendido'].rank(method='dense', ascending=False).astype(int)

# Exibir os 10 mais vendidos
print(ranking_df.head(10))

            produto  total_vendido  rank
0    Pasta De Dente         208234     1
1  Queijo Mussarela         206800     2
2          Sabonete         196673     3
3          Manteiga         192726     4
4      Papel Toalha         148968     5
5      Desinfetante         146369     6
6           Cerveja         122293     7
7   Molho De Tomate          99168     8
8     Condicionador          91084     9
9          Macarrão          76037    10


In [13]:
#agora vamos localizar o estado em que mais ocorre vendas
vendas_por_estado = df.groupby('estado')['quantidade'].sum().sort_values(ascending=False)

ranking_estados = vendas_por_estado.reset_index().rename(columns={'quantidade': 'total_vendido'})
ranking_estados['rank'] = ranking_estados['total_vendido'].rank(method='dense', ascending=False).astype(int)

print(ranking_estados.head(10))

  estado  total_vendido  rank
0     MG        1630441     1
1     RJ         709894     2
2     SP         279504     3
3     PR            931     4
4     SC            926     5
5     RS            891     6
6     MT            857     7


In [14]:
#Agora vamos ver qual é o melhor vendedor do estado de MG 

df_mg = df[df['estado'] == 'MG']


ranking_vendedores_mg = df_mg['vendedor'].value_counts().reset_index()
ranking_vendedores_mg.columns = ['vendedor', 'quantidade_vendas']


melhor_vendedor_mg = ranking_vendedores_mg.iloc[0]
print(f"Melhor Vendedor de MG: {melhor_vendedor_mg['vendedor']} com {melhor_vendedor_mg['quantidade_vendas']} vendas.")

Melhor Vendedor de MG: Micael Malaquias De Souza Oliveira com 22682 vendas.


In [15]:
# Filtra as regras com base no produto vendido pelo melhor vendedor
produtos_vendidos_melhor_vendedor = df_mg[df_mg['vendedor'] == melhor_vendedor_mg['vendedor']]['produto'].unique()
regras_com_produtos_vendedor = regras[regras['antecedents'].apply(lambda x: any(produto in x for produto in produtos_vendidos_melhor_vendedor))]

print(f"Regras de associação para produtos vendidos pelo melhor vendedor de MG:\n{regras_com_produtos_vendedor}")

Regras de associação para produtos vendidos pelo melhor vendedor de MG:
                                        antecedents  \
0                                       (Amaciante)   
1                                       (Amaciante)   
2                                           (Arroz)   
3                                 (Molho De Tomate)   
4                                           (Arroz)   
..                                              ...   
365            (Pasta De Dente, Sabonete, Manteiga)   
366    (Pasta De Dente, Queijo Mussarela, Manteiga)   
367  (Queijo Mussarela, Óleo De Soja, Pão De Forma)   
368          (Manteiga, Óleo De Soja, Pão De Forma)   
369                    (Óleo De Soja, Pão De Forma)   

                      consequents  antecedent support  consequent support  \
0                (Pasta De Dente)            0.074685            0.297405   
1                      (Sabonete)            0.074685            0.277738   
2                        (Feijão)   

In [20]:
# Filtra as vendas do melhor vendedor
df_vendas_melhor_vendedor = df_mg[df_mg['vendedor'] == melhor_vendedor_mg['vendedor']]

# Conta a quantidade de cada produto vendido por esse vendedor
top_5_produtos = df_vendas_melhor_vendedor['produto'].value_counts().head(5).reset_index()
top_5_produtos.columns = ['produto', 'quantidade_vendida']

print(f"\nTop 5 produtos mais vendidos por {melhor_vendedor_mg['vendedor']}:\n{top_5_produtos}")



Top 5 produtos mais vendidos por Micael Malaquias De Souza Oliveira:
            produto  quantidade_vendida
0  Queijo Mussarela                1646
1    Pasta De Dente                1580
2          Sabonete                1517
3          Manteiga                1488
4              Café                1428


In [22]:
# Salva as regras filtradas em um arquivo CSV dentro do diretório correto do Kaggle
output_path = "/kaggle/working/regras_associacao_top_5_produtos.csv"
regras_filtradas.to_csv(output_path, index=False)

# Mensagem confirmando que o arquivo foi salvo
print(f"\n📁 Arquivo 'regras_associacao_top_5_produtos.csv' salvo com sucesso em {output_path}!")


📁 Arquivo 'regras_associacao_top_5_produtos.csv' salvo com sucesso em /kaggle/working/regras_associacao_top_5_produtos.csv!
